In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

Google_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = 'gemini-pro', temperature = 0.5)


AIMessage(content="The meaning of life is a profound question that has been contemplated by philosophers, theologians, and individuals throughout history. There is no one definitive answer that is universally accepted, as the meaning of life can vary depending on personal beliefs, experiences, and perspectives. However, some common themes that emerge in discussions about the meaning of life include:\n\n* **Purpose and Fulfillment:** Many people find meaning in life by pursuing goals, passions, or activities that give them a sense of purpose and fulfillment. This could involve contributing to society, making a difference in the world, or simply living a life that is true to one's values and aspirations.\n\n* **Relationships and Connections:** Strong relationships with family, friends, and loved ones can provide a deep sense of meaning and belonging. Nurturing these relationships and investing in the well-being of others can bring joy, fulfillment, and a sense of purpose.\n\n* **Personal

In [22]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    '''
    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response time, or resource efficiency.
-   PO (Portability)  : Requirements related to compatibility across various devices or platforms.
-   SC (Scalability)  : Requirements on the system's ability to handle growth in users or workload.
-   SE (Security)  : Requirements ensuring data security, authorization, or protection from threats.
-   US (Usability)  : Requirements focused on user-friendliness, ease of use, or intuitive design.

---

Now classify this new requirement:{requirement}

Answer: category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US)
    '''
)

examples = [{
        "question": "The system shall display real-time data updates in a dashboard format accessible to all users.",
        "answer": "F"
    },
    {
        "question": "The product shall maintain 99.5% uptime during business hours, ensuring minimal service interruptions.",
        "answer": "A",
    },
    {
        "question": "The system shall continue to operate normally in case of a single hardware failure.",
        "answer": "FT",
    },
    {
        "question": "The application shall comply with GDPR regulations to ensure data privacy for all EU users.",
        "answer": "L",
    },
    {
        "question": "The product interface shall adhere to corporate branding guidelines, using the approved color palette and typography.",
        "answer": "LF",
    },
    {
        "question": "The system shall allow software updates to be applied without downtime for 95% of maintenance events.",
        "answer": "MN",
    },
    {
        "question": "The product shall support all versions of Windows and Mac operating systems released in the last five years.",
        "answer": "O",
    },
    {
        "question": "The system shall process and return search results within 3 seconds for 90% of user queries.",
        "answer": "PE",
    },
    {
        "question": "The application must be usable on both desktop and mobile devices without a loss of functionality.",
        "answer": "PO",
    },
    {
        "question": "The system shall handle up to 10,000 concurrent users without degradation of performance.",
        "answer": "SC",
    },
    {
        "question": "All user data shall be encrypted in transit and at rest to prevent unauthorized access.",
        "answer": "SE",
    },
    {
        "question": "The product shall allow 90% of first-time users to complete their initial setup within 5 minutes.,",
        "answer": "US",
    },
]

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import FewShotPromptTemplate

chain = LLMChain(
    llm= llm, 
    prompt = prompt_template,
    verbose = False
    )

requirement = "The user interface shall have standard menus  buttons for navigation"



prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt_template,
    suffix="Requirement: {input}",
    input_variables=["requirement"],
)

response = chain.invoke(input = requirement)

print(response['text'])




> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response time, or resourc

In [25]:
import pandas as pd

# Read the file
file_path = './data/test.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize lists to store separated data
classes = []
texts = []

# Process each line
for line in lines:
    # Split the line into class and text parts
    if ':' in line:
        class_part, text_part = line.split(":",1)
        classes.append(class_part.strip())
        texts.append(text_part.strip())

# Create a DataFrame from the lists
import pandas as pd

test = pd.DataFrame({
    'class': classes,
    'text': texts,
})

# Display the first few rows to verify
print(test)

   class                                               text
0      F     The system shall display Events or Activities.
1      F  The display shall have two regions:  left 2/3 ...
2      F  The data displayed in both the nodes within th...
3      F  The table side of the display shall be split i...
4      F  The top 1/4 of the table will hold events that...
..   ...                                                ...
65    SE  The Disputes application shall manipulate and ...
66    SE  All updates to data files or database must be ...
67     L  The Disputes application must conform to the l...
68     L  All business rules specified in the Disputes S...
69     L  The Disputes application must conform to the l...

[70 rows x 2 columns]


In [26]:
import time

output_of_prompt = []

i = 0
while i != len(test):
    try:
        output = chain.invoke(f"{test.iloc[i]['text']}")
        output_of_prompt.append({'i':i,
                                'text':test.iloc[i]['text'],
                                'labels':test.iloc[i]['class'],
                                'prompt_output': output})

        time.sleep(2)
        i += 1
        print(f'{i}/{len(test)} done :)')
    except Exception as e:
        print(f'\n sleep time by 10 seconds for {i}')
        print("Error: ",e)
        time.sleep(10)



> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response time, or resourc

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


16/70 done :)


> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response ti

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


32/70 done :)


> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response ti

...



> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response time, or resourc

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


68/70 done :)


> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response ti

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


69/70 done :)


> Entering new LLMChain chain...
Prompt after formatting:

    Classify the requirement text into one of the following categories by answering only with the category code (F, A, FT, L, LF, MN, O, PE, PO, SC, SE, or US):
    
-   F (Functional)  : Requirements detailing specific system functionalities or actions.
-   A (Availability)  : Requirements related to system uptime, accessibility, or continuous operation.
-   FT (Fault Tolerance)  : Requirements ensuring the system can handle errors or unexpected failures.
-   L (Legal)  : Requirements concerning compliance with laws, regulations, or industry standards.
-   LF (Look & Feel)  : Requirements about the appearance, design, color schemes, or visual style.
-   MN (Maintainability)  : Requirements on ease of updates, maintenance, or adjustments.
-   O (Operational)  : Requirements on system operations, such as supported platforms or environments.
-   PE (Performance)  : Requirements focused on system speed, response ti

In [27]:
output_of_prompt

[{'i': 0,
  'text': 'The system shall display Events or Activities.',
  'labels': 'F',
  'prompt_output': {'requirement': 'The system shall display Events or Activities.',
   'text': 'F'}},
 {'i': 1,
  'text': 'The display shall have two regions:  left 2/3 of the display is graphical  right 1/3 of the display is a data table',
  'labels': 'F',
  'prompt_output': {'requirement': 'The display shall have two regions:  left 2/3 of the display is graphical  right 1/3 of the display is a data table',
   'text': 'LF'}},
 {'i': 2,
  'text': 'The data displayed in both the nodes within the graph and the rows in the table are MSEL Summary data',
  'labels': 'F',
  'prompt_output': {'requirement': 'The data displayed in both the nodes within the graph and the rows in the table are MSEL Summary data',
   'text': 'F'}},
 {'i': 3,
  'text': 'The table side of the display shall be split into 2 regions: sequential and temporal.',
  'labels': 'F',
  'prompt_output': {'requirement': 'The table side of t

In [31]:
pred = []
label = []
data  = output_of_prompt
for d in data:
    pred.append(d['prompt_output']['text'])
    label.append(d['labels'])

In [34]:
import numpy as np
from collections import defaultdict

def perf_measure(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    if len(y_true) != len(y_pred):
        raise ValueError("The lengths of y_true and y_pred must match.")

    # Get the unique classes
    classes = np.unique(np.concatenate((y_true, y_pred)))

    # Initialize dictionaries to store results
    metrics = {cls: {"TP": 0, "FP": 0, "TN": 0, "FN": 0} for cls in classes}

    for cls in classes:
        metrics[cls]["TP"] = np.sum((y_true == cls) & (y_pred == cls))
        metrics[cls]["FP"] = np.sum((y_true != cls) & (y_pred == cls))
        metrics[cls]["TN"] = np.sum((y_true != cls) & (y_pred != cls))
        metrics[cls]["FN"] = np.sum((y_true == cls) & (y_pred != cls))

    TP = sum(values["TP"] for values in metrics.values())
    FP = sum(values["FP"] for values in metrics.values())
    TN = sum(values["TN"] for values in metrics.values())
    FN = sum(values["FN"] for values in metrics.values())
    return TP, FP, TN, FN

TP, FP, TN, FN = perf_measure(label, pred)

accuracy = (TP + TN) / (TP + FP + TN + FN)
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
F1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print('Accuracy: ', accuracy)
print('Recall: ', recall)
print('Precision: ', precision)
print('F1-score: ', F1)

Accuracy:  0.9469387755102041
Recall:  0.8142857142857143
Precision:  0.8142857142857143
F1-score:  0.8142857142857143
